In [3]:
import os
from huggingface_hub import InferenceClient

client = InferenceClient()

completion = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-V3-0324",
    messages=[
        {
            "role": "user",
            "content": "What date is it today ?"
        }
    ],
)

print(completion.choices[0].message)

ChatCompletionOutputMessage(role='assistant', content="Today's date is **June 4, 2024**.  \n\nLet me know if you'd like the time as well! 😊", tool_call_id=None, tool_calls=None)


In [4]:
!pip install transformers torch bitsandbytes accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 8.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Specify the model you want to use
model_id = "google/gemma-2b-it"#"deepseek-ai/DeepSeek-V3-0324"

# Load the tokenizer and model from the local cache (or download if not present)
# device_map="auto" will automatically use a GPU if available
# torch_dtype="auto" will select the best data type for your hardware
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto"
)

# Define the messages in the chat format the model expects
messages = [
    {"role": "user", "content": "Peux-tu écrire en français et me répondre en français ?"}
]

# Use the tokenizer to apply the chat template and convert to tensor format
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device) # Move the input tensor to the same device as the model

# Generate a response
# The model will output a sequence of token IDs
outputs = model.generate(
    input_ids,
    max_new_tokens=512,  # You can adjust the maximum length of the response
    eos_token_id=tokenizer.eos_token_id
)

# Decode the generated token IDs back into a string
response_ids = outputs[0][input_ids.shape[-1]:]
response = tokenizer.decode(response_ids, skip_special_tokens=True)

# Print the response
print(response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Oui, je peux écrire en français et répondre en français. Veuillez me donner un sujet ou une question en français, et je ferai de mon mieux pour répondre à votre question en français.
